In [ ]:
from ipyparallel import Client
client = Client()

In [ ]:
%%px
import functools as ft
import os, sys
sys.path.append(os.path.abspath('../../two_dim_majoranas/'))

import numpy as np
import scipy.constants
import cmath
# import holoviews as hv
# from IPython.core.magic import register_cell_magic
# import ipywidgets as widgets

import hpc05
import sns_system
# , plotting_results
import supercurrent_matsubara
# from distributed_sns import AggregatesSimulationSet as ASS
# from distributed_sns import SimulationSet as SS
# import adaptive
# adaptive.notebook_extension()

# run_cluster = widgets.Checkbox(
#     value=False,
#     description='Check to run cluster',
#     disabled=False)
# stop_cluster = widgets.Checkbox(
#     value=False,
#     description='Check to stop cluster',
#     disabled=False)

# @register_cell_magic
# def cluster_check(line, cell):
#     if run_cluster.value is True:
#         run_cluster.value = False
#         return line, cell
#     elif stop_cluster.value is True:
#         stop_cluster.value = False
#         return hpc05.kill_remote_ipcluster()
#     else:
#         return "Command not run. Check the box above to run."

# widgets.HBox([run_cluster, stop_cluster])

In [ ]:
%%cluster_check
client, dview, lview = hpc05.start_remote_and_connect(20, folder='~/two_dim_majoranas', timeout=180)

In [ ]:
%%px
constants = dict(
    m_eff=0.02 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    e = scipy.constants.e,
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )

In [ ]:
%%px
syst_pars = {'L_down' :  100,
               'L_up' : 100,
               'L_m' : 400,
               'L_x' : 800,
               'a' :  10,
               'mu_from_bottom_of_spin_orbit_bands': True,
               'with_vlead' : True}

params_raw= dict(g_factor_middle = 26,
                 g_factor_left = 0,
                 g_factor_right = 0,
                 mu = 20,
                 alpha_middle = 19.05,
                 alpha_left = 0,
                 alpha_right = 0,
                 Delta_left = 2,
                 Delta_right = 2,
                 B = 0.5,
                 phase = 0,
                 T = 0.1,
                 V = 0.0)

params = dict(**constants,
              **params_raw)

In [ ]:
%%px

def cur_B(B):
    _params = params.copy()
    _params['B'] = B
    syst, hopping = sns_system.make_sns_leaded_system(**syst_pars)
    el_blocks = ft.partial(sns_system.take_electron_blocks, norbs=4)
    I_c = supercurrent_matsubara.I_c(syst, hopping, _params, el_blocks, N_brute=101)
    return (I_c['phases'], I_c['currents']/syst_pars['L_x'])

In [ ]:
import numpy as np
Bs = np.linspace(0, 2, 24)
def cur_B(B):
    _params = params.copy()
    _params['B'] = B
    syst, hopping = sns_system.make_sns_leaded_system(**syst_pars)
    el_blocks = ft.partial(sns_system.take_electron_blocks, norbs=4)
    I_c = supercurrent_matsubara.I_c(syst, hopping, _params, el_blocks, N_brute=101)
    return (I_c['phases'], I_c['currents']/syst_pars['L_x'])

In [ ]:
lview = client.load_balanced_view()

In [ ]:
res = lview.map(cur_B, Bs)

In [ ]:
res.completed

In [ ]:
res.done()






In [ ]:
pc = res.result()

In [ ]:
phases=[]
currents=[]
for ph, curs in pc:
    phases.append(ph)
    currents.append(curs)

In [ ]:
Phases = np.vstack(phases)
Currents = np.vstack(currents)

In [ ]:
import holoviews as hv
hv.extension('bokeh')

In [ ]:
hv.Image((phases[0], Bs, Currents))

In [ ]:
hv.Image((phases[0], Bs, Currents))

In [ ]:
1.28/2

In [ ]:
hv.Image((phases[0], Bs, Currents))

In [ ]:
hv.Image((phases[0], Bs, Currents))

In [ ]:
import scipy.constants
import cmath 

constants = dict(
    m_eff=0.02 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    e = scipy.constants.e,
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )

syst_pars = {'L_down' :  100,
               'L_up' : 100,
               'L_m' : 400,
               'L_x' : 800,
               'a' :  10,
               'mu_from_bottom_of_spin_orbit_bands': True,
               'with_vlead' : True}

params_raw= dict(g_factor_middle = 26,
                 g_factor_left = 0,
                 g_factor_right = 0,
                 mu = 20,
                 alpha_middle = 19.05,
                 alpha_left = 0,
                 alpha_right = 0,
                 Delta_left = .2,
                 Delta_right = .2,
                 B = 0.5,
                 phase = 0,
                 T = 0.1,
                 V = 0.0)

params = dict(**constants,
              **params_raw)

In [ ]:
import os, sys
sys.path.append(os.path.abspath('../../two_dim_majoranas/'))
import formulas as f
vf = f.fermi_velocity(params['mu'], 0.02)
Eth = f.thouless_energy(vf, syst_pars['L_m'])
print(Eth)

In [ ]:
params['m_eff']*params['alpha_middle']**2/(2*params['hbar']**2)

In [ ]:
params['hbar'] * (np.pi/2) * vf/syst_pars['L_m'] /.64

In [ ]:
(params['hbar'] * vf/syst_pars['L_m'])